In [1]:
import sqlite3
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
conn = sqlite3.connect('../max-experiments/itinerary-scraping/journeys.db')
c = conn.cursor()

In [3]:
c.execute('SELECT * FROM journeys')
df = gpd.GeoDataFrame(c.fetchall(), columns=['id', 'total_duration', 'start_datetime', 'end_datetime', 'gec', 'nox_pm', 'method_1', 'method_2', 'path'])
df['gec'] = df['gec'].apply(lambda x: json.loads(x))
df['nox_pm'] = df['nox_pm'].apply(lambda x: json.loads(x))
df['path'] = df['path'].apply(lambda x: json.loads(x))
df['method_1'] = df['method_1'].apply(lambda x: json.loads(x))
df['method_2'] = df['method_2'].apply(lambda x: json.loads(x))
df.head()

,id,total_duration,start_datetime,end_datetime,gec,nox_pm,method_1,method_2,path
0,1,2972,20240525T175138,20240525T184110,"{'value': 933.1212, 'unit': 'gEC'}","{'unit': 'g', 'values': {'nox': 0, 'pm': 0}}","{'total': 2972, 'walking': 1232, 'bike': 0, 'c...","{'walking': 272, 'bike': 0, 'car': 0, 'ridesha...","[{'stop_point': 'Liberté', 'arrival_date_time'..."
1,2,2636,20240525T173733,20240525T182129,"{'value': 380.232, 'unit': 'gEC'}","{'unit': 'g', 'values': {'nox': 0, 'pm': 0}}","{'total': 2636, 'walking': 1436, 'bike': 0, 'c...","{'walking': 612, 'bike': 0, 'car': 0, 'ridesha...","[{'stop_point': 'Pont de La Mulatière', 'arriv..."
2,3,3025,20240525T174745,20240525T183810,"{'value': 558.0722, 'unit': 'gEC'}","{'unit': 'g', 'values': {'nox': 0, 'pm': 0}}","{'total': 3025, 'walking': 1945, 'bike': 0, 'c...","{'walking': 586, 'bike': 0, 'car': 0, 'ridesha...","[{'stop_point': 'Petit Versailles', 'arrival_d..."
3,4,3685,20240525T174745,20240525T184910,"{'value': 726.4134, 'unit': 'gEC'}","{'unit': 'g', 'values': {'nox': 0, 'pm': 0}}","{'total': 3685, 'walking': 1585, 'bike': 0, 'c...","{'walking': 586, 'bike': 0, 'car': 0, 'ridesha...","[{'stop_point': 'Petit Versailles', 'arrival_d..."
4,5,3685,20240525T174745,20240525T184910,"{'value': 705.801, 'unit': 'gEC'}","{'unit': 'g', 'values': {'nox': 0, 'pm': 0}}","{'total': 3685, 'walking': 1525, 'bike': 0, 'c...","{'walking': 586, 'bike': 0, 'car': 0, 'ridesha...","[{'stop_point': 'Petit Versailles', 'arrival_d..."


In [4]:
c.execute('SELECT * FROM stops')
stops_df = gpd.GeoDataFrame(c.fetchall(), columns=['stop_id', 'stop_name', 'lon_lat', 'accessibility'])
stops_df['lon_lat'] = stops_df['lon_lat'].apply(lambda x: json.loads(x))
stops_df.head()

,stop_id,stop_name,lon_lat,accessibility
0,stop_point:tcl:SP:32113,Liberté,"{'lon': '4.842381', 'lat': '45.758852'}","[""has_wheelchair_boarding""]"
1,stop_point:tcl:SP:32111,Guillotière Gabriel Péri,"{'lon': '4.842434', 'lat': '45.754935'}","[""has_wheelchair_boarding""]"
2,stop_point:tcl:SP:30200,Guillotière Gabriel Péri,"{'lon': '4.84252', 'lat': '45.75541'}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
3,stop_point:tcl:SP:30202,Bellecour,"{'lon': '4.833822', 'lat': '45.756711'}","[""has_wheelchair_boarding"", ""has_elevator"", ""h..."
4,stop_point:tcl:SP:45799,Bellecour Le Viste,"{'lon': '4.833739', 'lat': '45.757154'}","[""has_wheelchair_boarding""]"


In [ ]:
# calculate the distance of each journey
# https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
# returns the distance in meters
def coord_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Given two coordinates, returns the distance between them in meters.
    Args:
        lat1 (float): latitude of the first point
        lon1 (float): longitude of the first point
        lat2 (float): latitude of the second point
        lon2 (float): longitude of the second point
        
    Returns:
        float: distance between the two points in meters
    """
    R = 6371e3
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2) * np.sin(delta_phi / 2) + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2) * np.sin(delta_lambda / 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    d = R * c
    return d

distances = []
for i, row in df.iterrows():
    path = row['path']
    distance = 0
    for i in range(len(path) - 1):
        start = path[i]['stop_point']
        end = path[i+1]['stop_point']
        
        start_coords = stops_df[stops_df['stop_name'] == start]['lon_lat'].values[0]
        end_coords = stops_df[stops_df['stop_name'] == end]['lon_lat'].values[0]
        distance += coord_distance(float(start_coords['lon']), float(start_coords['lat']), float(end_coords['lon']), float(end_coords['lat']))
    distances.append(distance)
    
df['distance'] = distances

# save the data to a csv so I never have to do this again
df.to_csv('journeys.csv')

In [ ]:
# remove rows that are outliers in terms of total_duration
q1 = df['total_duration'].quantile(0.25)
q3 = df['total_duration'].quantile(0.75)
iqr = q3 - q1
df = df[(df['total_duration'] > (q1 - 1.5 * iqr)) & (df['total_duration'] < (q3 + 1.5 * iqr))]

In [ ]:
gEC = []
for row in df['gec']:
    gEC.append(row['value'])

print('gEC mean:', np.mean(gEC))
print('gEC median:', np.median(gEC))
print('gEC std:', np.std(gEC))

In [ ]:
# plot gEC vs total_duration
plt.scatter(df['total_duration'], gEC, alpha=0.5, s=10)
plt.xlabel('Total Trip Duration')
plt.ylabel('gEC')
plt.title('gEC vs Total Trip Duration')

# trendline with correlation coefficient
z = np.polyfit(df['total_duration'], gEC, 1)
p = np.poly1d(z)
plt.plot(df['total_duration'],p(df['total_duration']),"r--")
plt.xlim(500, 6500)
plt.savefig('out/gec/gEC_vs_total_duration.png')
print('Correlation coefficient:', round(np.corrcoef(df['total_duration'], gEC)[0, 1], 3))
print('Equation of trendline:', p)

In [ ]:
# histogram of gEC
plt.clf()
plt.hist(gEC, bins=50)
plt.xlabel('gEC')
plt.xlim(0, 3000)
plt.ylabel('Frequency')
plt.title('Histogram of gEC')
plt.savefig('out/gec/histogram_of_gEC.png')

In [ ]:
# plot gEC vs hour of day
start_times = []
for row in df['start_datetime']:
    # HHMMSS
    time = int(row.split('T')[1])
    time = time // 10000
    start_times.append(time)

In [ ]:
plt.scatter(start_times, gEC)
plt.xlabel('Hour of Day')
plt.ylabel('gEC')
plt.savefig('out/gec/gEC_vs_hour_of_day.png')
plt.show()

In [ ]:
# line graph of average gEC vs hour of day
avg_gEC = []
for i in range(5,24):
    avg_gEC.append(sum([gEC[j] for j in range(len(gEC)) if start_times[j] == i]) / len([gEC[j] for j in range(len(gEC)) if start_times[j] == i]))
    
plt.plot(range(5,24), avg_gEC, marker='o')
plt.xlabel('Hour of Day')
plt.ylabel('gEC')
plt.title('Average gEC vs Hour of Day')
plt.ylim(0, 1000)
plt.savefig('out/gec/avg_gEC_vs_hour_of_day.png')
plt.show()